# 1) Scraping des données

Tout d'abord, il nous a fallu récupérer les données utiles des hôtels parisiens sur le site Booking.com. Nous nous sommes rendus compte que ces informations n'étaient pas directement accessibles via le code source, mais qu'elles l'étaient avec la fonction "Inspecter l'élément" de nos navigateurs Internet. Cela est certainement dû à l'utilisation par le site de scripts Java.

Nous avons donc créé une fonction qui ouvre une page Web grâce à Selenium, puis qui cherche sur cette page l'ensemble des entrées correspondant aux classes contenant les informations utiles. Ces entrées sont sauvegardées dans des tableaux. Une fois les données recueillies, le navigateur se ferme.

In [ ]:
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import numpy as np

Dans tout le notebook, à chaque fois que la commande "driver = webdriver.Chrome()" est utilisée, on peut également utiliser la commande "driver = webdriver.Chrome(ChromeDriverManager().install())". En effet sur certains ordinateurs la première option ne fonctionne pas toujours. Pour utiliser la deuxième option, il faut exécuter la cellule ci dessous : 

In [ ]:
#option 2 (si vous avez des problèmes de webdriver ce qui dépend de l'ordinateur, voir partie 1.1.1 ci dessous)
!pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager

Nous souhaitons utiliser cette fonction sur les différentes pages du site afin de disposer d'une base de données suffisamment importante. Nous nous sommes rendus compte que la page $i+1$ est accessible en ajoutant "&offset25$(i+1)$" à l'URL de la page. De fait, nous avons créé une liste avec les 40 premières URL (de manière à obtenir une base avec 1 000 hôtels car il y a 25 hôtels par page).

In [59]:
def cree_liste(url):
    liste=[]
    for i in range(40):
        if i==0:
            liste.append(url)
        else:
            liste.append(url+'&offset='+str(25*i))
    return(liste)

# 1.1. Web-scraping pour l'évolution des prix

Afin de pouvoir comparer les prix, il faut pouvoir changer de jour de réservation de façon automatique. Les jours (pour un mois donné) de réservation sont indiqués dans l'URL grâce à la référence "checkout_monthday".

In [19]:
jours = [(a,11) for a in range(16,31)]+[(a,12) for a in range(1,32)]
jours[:3]

[(16, 11), (17, 11), (18, 11)]

### 1.1.1. Scrapping 

In [21]:
print(prix[:1], noms_hotels[:1])

[['€ 129', '€ 133', '€ 126', '€ 87', '€ 246', '€ 89', '€ 156', '€ 142', '€ 155', '€ 133', '€ 221', '€ 107', '€ 143', '€ 196', '€ 176', '€ 120', '€ 131', '€ 116', '€ 136', '€ 142', '€ 183', '€ 145', '€ 62', '€ 148', '€ 74']] [['Acacias Etoile', 'Holiday Inn Paris Montmartre, an IHG Hotel', 'Hôtel Paris La Fayette', "Mary's Hotel République", 'Hotel du Petit Moulin', "Hotel de l'Aqueduc", 'Petit Madeleine Hôtel', 'Elysées Union', 'B Montmartre', 'Hotel Des Deux Continents', 'Millésime Hôtel', 'Welcome Hotel', 'Auteuil Tour Eiffel', 'Les Tournelles', 'Hôtel Saint-Pétersbourg Opéra & Spa', 'Hotel De Seine', 'Hotel Berne Opera', 'Hotel Sacha', "Hotel d'Espagne", 'Hôtel Bleu de Grenelle', 'Mercure Paris Opera Grands Boulevards', 'Hotel Trianon Rive Gauche', 'MEININGER Hotel Paris Porte de Vincennes', 'Montfleuri', 'Virgina']]


In [ ]:
prix = []
noms_hotels = []
# Pour chaque jour, on change l'URL puis on récupère les prix sur les 40 pages
for jour in range(len(jours)-1): 
    
    #url valide jusqu'au 31/01/2022:
    url='https://www.booking.com/searchresults.fr.html?aid=356980&label=gog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE&sid=c81cc6cca40ba1a705208d4213ee6868&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D356980%3Blabel%3Dgog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE%3Bsid%3Dc81cc6cca40ba1a705208d4213ee6868%3Btmpl%3Dsearchresults%3Bcity%3D-1456928%3Bclass_interval%3D1%3Bdest_id%3D-1456928%3Bdest_type%3Dcity%3Blabel_click%3Dundef%3Boffset%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrpvid%3Db13d753bb2110028%3Bssb%3Dempty%26%3B&ss=Paris&is_ski_area=0&ssne=Paris&ssne_untouched=Paris&city=-1456928&checkin_year=2022&checkin_month=2&checkin_monthday=1&checkout_year=2022&checkout_month=2&checkout_monthday=2&group_adults=2&group_children=0&no_rooms=1&from_sf=1'
    
    #url réellement utilisée (mais qui n'est plus valide actuellement):
    #url = 'https://www.booking.com/searchresults.fr.html?label=gog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE&sid=81cd22762f2169ba954d21a297ff0a64&aid=356980&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D356980%3Blabel%3Dgog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE%3Bsid%3D81cd22762f2169ba954d21a297ff0a64%3Btmpl%3Dsearchresults%3Bcity%3D-1456928%3Bclass_interval%3D1%3Bdest_id%3D-1456928%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrpvid%3D227a56f9e7060099%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=Paris&is_ski_area=0&ssne=Paris&ssne_untouched=Paris&city=-1456928&checkin_year=2021&checkin_month=11&checkin_monthday=6&checkout_year=2021&checkout_month=11&checkout_monthday=7&group_adults=2&group_children=0&no_rooms=1&sb_changed_dates=1&from_sf=1&nflt=ht_id%3D204'
    
    #A chaque fois, on remet l'URL de départ. Cette solution n'est pas optimale en termes de temps de calcul, mais évite d'écrire des lignes supplémentaires car il faudrait s'adapter à l'URL qui est changée à chaque itération.
    url = url.replace('checkin_year=2021&checkin_month=11&checkin_monthday=6&checkout_year=2021&checkout_month=11&checkout_monthday=7','checkin_year=2021&checkin_month='+str(jours[jour][1])+'&checkin_monthday='+str(jours[jour][0])+'&checkout_year=2021&checkout_month='+str(jours[jour+1][1])+'&checkout_monthday='+str(jours[jour+1][0]))

    liste = cree_liste(url)
    for i in range(4):
    
        #option 1:
        #driver = webdriver.Chrome()
        
        #option 2:
        driver = webdriver.Chrome(ChromeDriverManager().install())
        
        driver.get(liste[i])
        noms_hotels.append([hotel.text for hotel in wait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fde444d7ef._c445487e2")))])
        prix.append([prix.text for prix in wait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fde444d7ef._e885fdc12")))])
        driver.quit()

On obtient bien les noms des hôtels ainsi que les prix pour les différentes dates choisies. Les dates sont ajoutées dans la section suivante. 

### 1.1.2. mise en forme des données

In [26]:
prix2 = {}
for x in range(16, 31):
    prix2["{0}/11/2021".format(x)] = prix[(x-16)*4]+prix[(x-16)*4 +1]+prix[(x-16)*4 +2]+prix[(x-16)*4 +3] #-1+(x-15)*4# prix[(x-16)*4:-1+(x-15)*4]
for x in range(1, 31):
    prix2["{0}/12/2021".format(x)] = prix[60+(x-1)*4]+prix[61+(x-1)*4]+prix[62+(x-1)*4]+prix[63+(x-1)*4]
print(prix2)

nom_hotel_2 = {}
for x in range(16, 31):
    nom_hotel_2["{0}/11/2021".format(x)] = noms_hotels[(x-16)*4]+noms_hotels[(x-16)*4 +1]+noms_hotels[(x-16)*4 +2]+noms_hotels[(x-16)*4 +3] #-1+(x-15)*4# prix[(x-16)*4:-1+(x-15)*4]
for x in range(1, 31):
    nom_hotel_2["{0}/12/2021".format(x)] = noms_hotels[60+(x-1)*4]+noms_hotels[61+(x-1)*4]+noms_hotels[62+(x-1)*4]+noms_hotels[63+(x-1)*4]
print(nom_hotel_2)

{'16/11/2021': ['€ 129', '€ 133', '€ 126', '€ 87', '€ 246', '€ 89', '€ 156', '€ 142', '€ 155', '€ 133', '€ 221', '€ 107', '€ 143', '€ 196', '€ 176', '€ 120', '€ 131', '€ 116', '€ 136', '€ 142', '€ 183', '€ 145', '€ 62', '€ 148', '€ 74', '€ 62', '€ 148', '€ 185', '€ 74', '€ 111', '€ 109', '€ 165', '€ 125', '€ 130', '€ 189', '€ 74', '€ 94', '€ 154', '€ 113', '€ 130', '€ 101', '€ 93', '€ 178', '€ 96', '€ 84', '€ 149', '€ 211', '€ 128', '€ 94', '€ 103', '€ 110', '€ 61', '€ 107', '€ 145', '€ 90', '€ 124', '€ 99', '€ 115', '€ 122', '€ 92', '€ 196', '€ 239', '€ 102', '€ 96', '€ 99', '€ 143', '€ 118', '€ 135', '€ 93', '€ 248', '€ 149', '€ 87', '€ 99', '€ 98', '€ 114', '€ 93', '€ 134', '€ 149', '€ 87', '€ 99', '€ 98', '€ 114', '€ 128', '€ 154', '€ 103', '€ 84', '€ 112', '€ 205', '€ 141', '€ 103', '€ 99', '€ 245', '€ 102', '€ 140', '€ 136', '€ 165', '€ 99', '€ 166', '€ 123', '€ 143'], '17/11/2021': ['€ 129', '€ 133', '€ 126', '€ 87', '€ 234', '€ 89', '€ 142', '€ 210', '€ 135', '€ 133', '€ 107', 

In [27]:
testdfprix = pd.DataFrame.from_dict(prix2)
testdfprix

,16/11/2021,17/11/2021,18/11/2021,19/11/2021,20/11/2021,21/11/2021,22/11/2021,23/11/2021,24/11/2021,25/11/2021,...,21/12/2021,22/12/2021,23/12/2021,24/12/2021,25/12/2021,26/12/2021,27/12/2021,28/12/2021,29/12/2021,30/12/2021
0,€ 129,€ 129,€ 129,€ 129,€ 129,€ 129,€ 198,€ 111,€ 129,€ 129,...,€ 129,€ 129,€ 129,€ 129,€ 129,€ 129,€ 129,€ 129,€ 129,€ 129
1,€ 133,€ 133,€ 133,€ 133,€ 133,€ 133,€ 133,€ 133,€ 133,€ 133,...,€ 133,€ 133,€ 133,€ 133,€ 133,€ 133,€ 133,€ 133,€ 133,€ 133
2,€ 126,€ 126,€ 126,€ 126,€ 126,€ 126,€ 126,€ 126,€ 83,€ 126,...,€ 87,€ 126,€ 126,€ 126,€ 126,€ 126,€ 126,€ 126,€ 126,€ 126
3,€ 87,€ 87,€ 83,€ 80,€ 80,€ 80,€ 80,€ 80,€ 126,€ 87,...,€ 126,€ 80,€ 87,€ 80,€ 87,€ 87,€ 83,€ 80,€ 80,€ 80
4,€ 246,€ 234,€ 234,€ 234,€ 234,€ 234,€ 246,€ 89,€ 246,€ 246,...,€ 234,€ 246,€ 234,€ 246,€ 246,€ 234,€ 234,€ 234,€ 234,€ 234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,€ 165,€ 141,€ 123,€ 124,€ 103,€ 141,€ 141,€ 109,€ 165,€ 110,...,€ 114,€ 114,€ 109,€ 114,€ 153,€ 119,€ 193,€ 119,€ 114,€ 99
96,€ 99,€ 103,€ 83,€ 166,"€ 91,05",€ 103,€ 103,€ 119,€ 175,€ 114,...,€ 119,€ 119,€ 119,€ 119,€ 115,€ 133,€ 138,€ 64,€ 119,€ 245
97,€ 166,"€ 91,05",€ 175,€ 123,€ 245,"€ 91,05",€ 78,€ 64,€ 123,€ 119,...,€ 133,€ 133,€ 110,€ 193,€ 133,€ 129,€ 87,€ 110,€ 133,€ 122
98,€ 123,€ 245,€ 119,€ 83,€ 126,€ 245,"€ 91,05",€ 110,€ 144,€ 133,...,€ 128,€ 128,€ 114,€ 138,€ 85,€ 128,€ 88,€ 114,€ 128,€ 140


In [28]:
testdfhotel = pd.DataFrame.from_dict(nom_hotel_2)
testdfhotel

,16/11/2021,17/11/2021,18/11/2021,19/11/2021,20/11/2021,21/11/2021,22/11/2021,23/11/2021,24/11/2021,25/11/2021,...,21/12/2021,22/12/2021,23/12/2021,24/12/2021,25/12/2021,26/12/2021,27/12/2021,28/12/2021,29/12/2021,30/12/2021
0,Acacias Etoile,Acacias Etoile,Acacias Etoile,Acacias Etoile,Acacias Etoile,Acacias Etoile,Hotel Prelude Opera,Hôtel Joséphine by Happyculture,Acacias Etoile,Acacias Etoile,...,Acacias Etoile,Acacias Etoile,Acacias Etoile,Acacias Etoile,Acacias Etoile,Acacias Etoile,Acacias Etoile,Acacias Etoile,Acacias Etoile,Acacias Etoile
1,"Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel",...,"Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel","Holiday Inn Paris Montmartre, an IHG Hotel"
2,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Mary's Hotel République,Hôtel Paris La Fayette,...,Mary's Hotel République,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette,Hôtel Paris La Fayette
3,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République,Hôtel Paris La Fayette,Mary's Hotel République,...,Hôtel Paris La Fayette,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République,Mary's Hotel République
4,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel de l'Aqueduc,Hotel du Petit Moulin,Hotel du Petit Moulin,...,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin,Hotel du Petit Moulin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Citadines Tour Eiffel Paris,Best Western Premier Kapital Opéra,Hotel des Grandes Ecoles,Maison Axel Opéra,Hôtel Brittany,Best Western Premier Kapital Opéra,Best Western Premier Kapital Opéra,Hôtel Elixir,Citadines Tour Eiffel Paris,Hotel des Arts,...,Austin's Saint Lazare Hotel,Austin's Saint Lazare Hotel,Hôtel Elixir,Austin's Saint Lazare Hotel,Hotel Elysees Opera,Hotel 34B - Astotel,Hotel Atmospheres,Hotel Armoni Paris,Austin's Saint Lazare Hotel,Elysees Niel Hotel
96,Hôtel Le Relais Saint Charles,Hôtel Brittany,Best Western Plus 61 Paris Nation Hotel,Hôtel Aiglon,Elysees Niel Hotel,Hôtel Brittany,Hôtel Brittany,Hotel Armoni Paris,Hôtel Aiglon,Austin's Saint Lazare Hotel,...,Hotel 34B - Astotel,Hotel 34B - Astotel,Hotel Armoni Paris,Hotel 34B - Astotel,Hotel Elysée Secret,Best Western Au Trocadéro,Chambellan Morgane,Hôtel Du Mont Dore Batignolles,Hotel 34B - Astotel,Le Pradey
97,Hôtel Aiglon,Elysees Niel Hotel,Les Jardins Du Luxembourg,Hotel des Grandes Ecoles,Le Pradey,Elysees Niel Hotel,Hôtel de Bordeaux,Hôtel Du Mont Dore Batignolles,

In [29]:
testdfprixT = testdfprix.melt()
testdfprixT = testdfprixT.rename(columns={"variable":"Date", "value":"Prix"})
testdfprixT

,Date,Prix
0,16/11/2021,€ 129
1,16/11/2021,€ 133
2,16/11/2021,€ 126
3,16/11/2021,€ 87
4,16/11/2021,€ 246
...,...,...
4495,30/12/2021,€ 99
4496,30/12/2021,€ 245
4497,30/12/2021,€ 122
4498,30/12/2021,€ 140


In [30]:
testdfhotelT = testdfhotel.melt()
testdfhotelT = testdfhotelT.rename(columns={"variable":"Date", "value":"Hôtel"})
testdfhotelT

,Date,Hôtel
0,16/11/2021,Acacias Etoile
1,16/11/2021,"Holiday Inn Paris Montmartre, an IHG Hotel"
2,16/11/2021,Hôtel Paris La Fayette
3,16/11/2021,Mary's Hotel République
4,16/11/2021,Hotel du Petit Moulin
...,...,...
4495,30/12/2021,Elysees Niel Hotel
4496,30/12/2021,Le Pradey
4497,30/12/2021,Timhotel Montmartre
4498,30/12/2021,COQ Hotel Paris


In [32]:
dfprix = pd.concat([testdfprixT, testdfhotelT], axis=1)
dfprix = dfprix.loc[:,~dfprix.columns.duplicated()] # Supprime la deuxième colonne "Date" qui est dupliquée
dfprix

,Date,Prix,Hôtel
0,16/11/2021,€ 129,Acacias Etoile
1,16/11/2021,€ 133,"Holiday Inn Paris Montmartre, an IHG Hotel"
2,16/11/2021,€ 126,Hôtel Paris La Fayette
3,16/11/2021,€ 87,Mary's Hotel République
4,16/11/2021,€ 246,Hotel du Petit Moulin
...,...,...,...
4495,30/12/2021,€ 99,Elysees Niel Hotel
4496,30/12/2021,€ 245,Le Pradey
4497,30/12/2021,€ 122,Timhotel Montmartre
4498,30/12/2021,€ 140,COQ Hotel Paris


On peut convertir la date au format "datetime" préconisé par Python, grâce à la fonction "to_datetime".

In [37]:
dfprix['Date'] = pd.to_datetime(dfprix['Date'],format='%d/%m/%Y')
dfprix

,Date,Prix,Hôtel
0,2021-11-16,129.0,Acacias Etoile
1,2021-11-16,133.0,"Holiday Inn Paris Montmartre, an IHG Hotel"
2,2021-11-16,126.0,Hôtel Paris La Fayette
3,2021-11-16,87.0,Mary's Hotel République
4,2021-11-16,246.0,Hotel du Petit Moulin
...,...,...,...
4495,2021-12-30,99.0,Elysees Niel Hotel
4496,2021-12-30,245.0,Le Pradey
4497,2021-12-30,122.0,Timhotel Montmartre
4498,2021-12-30,140.0,COQ Hotel Paris


Afin d'effectuer des calculs sur la variable "Prix", nous devons transformer le format de cette variable. Pour cela, nous enlevons le caractère "€" ainsi que l'espace qui le suit, puis nous convertirons cette variable au format numérique.

In [38]:
dfprix["Prix"] = dfprix["Prix"].replace('[€ ]', '')
dfprix["Prix"] = dfprix["Prix"].replace(',', '.')
dfprix["Prix"] = pd.to_numeric(dfprix["Prix"])
dfprix

,Date,Prix,Hôtel
0,2021-11-16,129.0,Acacias Etoile
1,2021-11-16,133.0,"Holiday Inn Paris Montmartre, an IHG Hotel"
2,2021-11-16,126.0,Hôtel Paris La Fayette
3,2021-11-16,87.0,Mary's Hotel République
4,2021-11-16,246.0,Hotel du Petit Moulin
...,...,...,...
4495,2021-12-30,99.0,Elysees Niel Hotel
4496,2021-12-30,245.0,Le Pradey
4497,2021-12-30,122.0,Timhotel Montmartre
4498,2021-12-30,140.0,COQ Hotel Paris


Enfin, on sauvegarde cette version du data-frame au format xlsx :

In [ ]:
#dfprix.to_excel('dfprix.xlsx')

### 1.1.3. valeurs manquantes

On s'est rendu compte en changeant de date de réservation que les hôtels qui s'affichaient n'étaient pas dans le même ordre. Sachant qu'on s'est contenté des 4 premières pages pour chaque jour de réservation, on pouvait s'imaginer que les prix de réservation pour certains hôtels à certains jours ne seraient pas disponibles. La fonction suivante permet de connaître, pour chaque hôtel, à la fois le nombre de jours pour lesquels les prix de réservation figurent dans notre base de données, ainsi que le nombre d'hôtels pour lesquels il y a au moins un prix de réservation répertorié.

In [44]:
dfprix.groupby("Hôtel")["Prix"].count()

Hôtel
55 Hôtel Montparnasse              43
9Hotel Bastille-Lyon               42
9Hotel Republique                  44
Aberotel Montparnasse Eiffel       45
Absolute Hotel Paris République    46
                                   ..
Tipi                               44
Trianon Gare de Lyon               46
Victoria Palace Hotel              42
Virgina                            43
Welcome Hotel                      43
Name: Prix, Length: 131, dtype: int64

In [45]:
np.mean(dfprix.groupby("Hôtel")["Prix"].count())

34.35114503816794

On peut constater que nous disposons d'une base de données avec 131 hôtels, alors qu'idéalement on en aurait eu 1000. De plus, en moyenne, nous avons plus de 34 prix répertoriés par hôtel, ce qui est assez satisfaisant. 

# 1.2. Web-scraping pour les caractéristiques des hôtels

## 1.2.1. Scraping de la base de données "Hôtels"

Dans cette partie, nous récupérons les données relatives aux caractéristiques des hôtels qui sont directement accessibles sur le moteur de recherche des hôtels (25 hôtels par page) : le nom, la note moyenne, le nombre d'avis de clients, le prix de réservation pour 2 personnes et 1 nuit, et l'arrondissement.

In [100]:
def recup_data(url):
    global hotels,note_moy,nb_avis,prix,arr
    driver = webdriver.Chrome()
    driver.get(url)

    # L'instruction ci-dessous demande à Python de chercher tous les éléments de la classe correspondant aux noms des hôtels, avec une limite de temps afin d'éviter un plantage du code en cas d'échec.

    #print([hotel.text for hotel in wait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fde444d7ef._c445487e2")))])
    hotels += [hotel.text for hotel in wait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fde444d7ef._c445487e2")))]
    #print(hotels)

    note_moy += [note.text for note in wait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "_9c5f726ff.bd528f9ea6")))]
    #print(note_moy)

    nb_avis += [nbavis.text for nbavis in wait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "_4abc4c3d5._1e6021d2f._6e869d6e0")))]
    #print(nb_avis) _4abc4c3d5 _1e6021d2f _6e869d6e0

    # Prix = prix avec promotion.
    prix += [prix.text for prix in wait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fde444d7ef._e885fdc12")))]
    #print(prix)

    arr2 = [arr.text for arr in wait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "af1ddfc958.eba89149fb")))]
    arr += [elem for elem in arr2 if elem != 'Indiquer sur la carte']
    #print(arr)

    # Page 1 = https://www.booking.com/searchresults.fr.html?aid=356980&label=gog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE&sid=81cd22762f2169ba954d21a297ff0a64&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D356980%3Blabel%3Dgog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE%3Bsid%3D81cd22762f2169ba954d21a297ff0a64%3Btmpl%3Dsearchresults%3Bcity%3D-1456928%3Bclass_interval%3D1%3Bdest_id%3D-1456928%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrpvid%3D227a56f9e7060099%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=Paris&is_ski_area=0&ssne=Paris&ssne_untouched=Paris&city=-1456928&checkin_year=2021&checkin_month=11&checkin_monthday=6&checkout_year=2021&checkout_month=11&checkout_monthday=7&group_adults=2&group_children=0&no_rooms=1&sb_changed_dates=1&from_sf=1
    # Page 2 = https://www.booking.com/searchresults.fr.html?label=gog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE&sid=81cd22762f2169ba954d21a297ff0a64&aid=356980&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D356980%3Blabel%3Dgog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE%3Bsid%3D81cd22762f2169ba954d21a297ff0a64%3Btmpl%3Dsearchresults%3Bcity%3D-1456928%3Bclass_interval%3D1%3Bdest_id%3D-1456928%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrpvid%3D227a56f9e7060099%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=Paris&is_ski_area=0&ssne=Paris&ssne_untouched=Paris&city=-1456928&checkin_year=2021&checkin_month=11&checkin_monthday=6&checkout_year=2021&checkout_month=11&checkout_monthday=7&group_adults=2&group_children=0&no_rooms=1&sb_changed_dates=1&from_sf=1&offset=25

    driver.quit()
    return(hotels, note_moy, nb_avis, arr)

À l'aide d'une boucle, nous pouvons désormais récupérer les données des hôtels pour 40 pages de résulats. 

Enfin, nous pouvons synthétiser l'ensemble des données obtenues dans un dataframe. Ce format nous permettra d'exploiter plus facilement ces données.

In [102]:
dfhotels = pd.DataFrame({"Hôtels":hotels,"Note moyenne":note_moy,"Nombre de commentaires":nb_avis,"Prix":prix,"Arrondissement":arr})
dfhotels

,Hôtels,Note moyenne,Nombre de commentaires,Prix,Arrondissement
0,Acacias Etoile,"8,4",849 expériences vécues,€ 129,"17e arr., Paris"
1,"Holiday Inn Paris Montmartre, an IHG Hotel","8,4",610 expériences vécues,€ 133,"18e arr., Paris"
2,Hôtel Paris La Fayette,"7,8",1 172 expériences vécues,€ 126,"10e arr., Paris"
3,Mary's Hotel République,"7,6",2 525 expériences vécues,€ 83,"11e arr., Paris"
4,Hotel du Petit Moulin,"8,8",187 expériences vécues,€ 234,"3e arr., Paris"
...,...,...,...,...,...
995,Le Mathurin Hotel & Spa,"8,5",1 176 expériences vécues,€ 216,"8e arr., Paris"
996,La Villa Haussmann,"8,9",1 147 expériences vécues,€ 273,"8e arr., Paris"
997,Hotel des Académies et des Arts,"8,8",271 expériences vécues,€ 195,"6e arr., Paris"
998,Hotel Elysée Gare de Lyon,"8,3",939 expériences vécues,"€ 140,86","12e arr., Paris"


In [101]:
hotels = []
note_moy = []
nb_avis = []
arr = []
prix = []

url = 'https://www.booking.com/searchresults.fr.html?aid=356980&label=gog235jc-1DCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBA-gBAfgBAogCAagCA7gC9MjXjgbAAgHSAiRkYWI0ODE5My1iYmM2LTQ5MmMtODNlZC04NThjODQzNDAzNzPYAgTgAgE&sid=766e7cd2b7bcbe2786949611337553bc&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D356980%3Blabel%3Dgog235jc-1DCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBA-gBAfgBAogCAagCA7gC9MjXjgbAAgHSAiRkYWI0ODE5My1iYmM2LTQ5MmMtODNlZC04NThjODQzNDAzNzPYAgTgAgE%3Bsid%3D766e7cd2b7bcbe2786949611337553bc%3Btmpl%3Dsearchresults%3Bcheckin_month%3D1%3Bcheckin_monthday%3D20%3Bcheckin_year%3D2022%3Bcheckout_month%3D1%3Bcheckout_monthday%3D21%3Bcheckout_year%3D2022%3Bcity%3D-1456928%3Bclass_interval%3D1%3Bdest_id%3D-1456928%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dcity%3Bsrc_elem%3Dsb%3Bsrpvid%3D4ebd828d9a5c00f7%3Bss%3DParis%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DParis%3Bssne_untouched%3DParis%26%3B&ss=Paris&is_ski_area=0&ssne=Paris&ssne_untouched=Paris&city=-1456928&checkin_year=2022&checkin_month=2&checkin_monthday=1&checkout_year=2022&checkout_month=2&checkout_monthday=2&group_adults=2&group_children=0&no_rooms=1&from_sf=1'
liste = cree_liste(url)

for i in range(40):
    recup_data(liste[i])

Afin de pouvoir exploiter ce dataframe, nous souhaitons supprimer les éventuels doublons et valeurs manquantes.

In [103]:
dfhotels = dfhotels.drop_duplicates()
dfhotels = dfhotels.dropna()
dfhotels

,Hôtels,Note moyenne,Nombre de commentaires,Prix,Arrondissement
0,Acacias Etoile,"8,4",849 expériences vécues,€ 129,"17e arr., Paris"
1,"Holiday Inn Paris Montmartre, an IHG Hotel","8,4",610 expériences vécues,€ 133,"18e arr., Paris"
2,Hôtel Paris La Fayette,"7,8",1 172 expériences vécues,€ 126,"10e arr., Paris"
3,Mary's Hotel République,"7,6",2 525 expériences vécues,€ 83,"11e arr., Paris"
4,Hotel du Petit Moulin,"8,8",187 expériences vécues,€ 234,"3e arr., Paris"
...,...,...,...,...,...
990,1er Etage Marais,"9,0",192 expériences vécues,€ 234,"4e arr., Paris"
993,Blc Design Hotel,"7,9",521 expériences vécues,€ 107,"11e arr., Paris"
994,Victoires Opera,"7,9",369 expériences vécues,€ 168,"2e arr., Paris"
998,Hotel Elysée Gare de Lyon,"8,3",939 expériences vécues,"€ 140,86","12e arr., Paris"


Nous nous rendons compte qu'il y avait plus de 200 doublons parmi les hôtels. Maintenons, nous sauvegardons le dataframe sous forme de fichier Excel. Cela nous permettra de retrouver nos données scrapées sans avoir à réexécuter l'ensemble du code.

In [ ]:
#dfhotels.to_excel('dfhotels.xlsx')

#### Ajout du nombre d'étoiles

On cherche à rajouter à ces données le nombre d'étoiles de chaque hôtel. Cette information ne peut pas être scrapée facilement à partir du site Booking.com, car les étoiles s'affichent sous forme d'images. On essaye donc de récupérer l'information à partir d'un script, qui consiste à :

1) chercher sur Google le nom de l'hôtel,

2) récupérer le nombre d'étoiles sur la page.

In [63]:
from selenium.common.exceptions import NoSuchElementException

In [64]:
def recup_etoiles(hotels):
    etoiles = []
    #option1:
    #driver = webdriver.Chrome()
    #option2:
    driver = webdriver.Chrome(ChromeDriverManager().install())
    url = 'https://www.google.com/search?q=étoiles hotel '
    driver.get(url)
    for hotel in hotels:
        driver.get(url+str(hotel))
        # Test 1 : regarder partout sur la page si le nombre d'étoiles apparaît sous forme de texte
        if len(driver.find_elements(By.XPATH, "//*[contains(text(),'5 étoiles')]"))!=0:
            etoiles.append(5)
        elif len(driver.find_elements(By.XPATH, "//*[contains(text(),'4 étoiles')]"))!=0:
            etoiles.append(4)
        elif len(driver.find_elements(By.XPATH, "//*[contains(text(),'3 étoiles')]"))!=0:
            etoiles.append(3)
        elif len(driver.find_elements(By.XPATH, "//*[contains(text(),'2 étoiles')]"))!=0:
            etoiles.append(2)
        elif len(driver.find_elements(By.XPATH, "//*[contains(text(),'1 étoile')]"))!=0:
            etoiles.append(1)
        else:
            # Test 1 : regarder si le bandeau de Google (à droite) affiche le nombre d'étoiles
            try:
                texte = driver.find_element(By.CLASS_NAME, "YhemCb").text
                if '5' in texte:
                    etoiles.append(5)
                elif '4' in texte:
                    etoiles.append(4)
                elif '3' in texte:
                    etoiles.append(3)
                elif '2' in texte:
                    etoiles.append(2)
                elif '1' in texte:
                    etoiles.append(1)
            # Sinon, le nombre d'étoiles n'apparaît pas sous forme de chaîne de caractères simple. On affiche alors "NA".
            except NoSuchElementException:
                etoiles.append(np.nan)
    return etoiles

Dans les faits, Google nous empêche d'effectuer trop de recherches d'affilée. Sinon, le moteur de recherche demande s'il a affaire à un robot ou non, et nous ne pouvons alors pas accéder au résultat de la recherche. On décide donc d'appliquer la fonction à notre base de données en plusieurs temps.

In [ ]:
etoiles1 = recup_etoiles(dfhotels['Hôtels'][0:200])

In [ ]:
etoiles2 = recup_etoiles(dfhotels['Hôtels'][200:400])

In [ ]:
etoiles3 = recup_etoiles(dfhotels['Hôtels'][400:600])

In [ ]:
etoiles4 = recup_etoiles(dfhotels['Hôtels'][600:len(dfhotels['Hôtels'])+1])

In [80]:
print(len(etoiles1),etoiles4[:5])

200 [4, 4, 4, 5, 4]


In [81]:
dfhotels['Etoiles']=etoiles1+etoiles2+etoiles3+etoiles4
dfhotels.head()

,Hôtels,Note moyenne,Nombre de commentaires,Prix,Arrondissement,Etoiles
0,Vice Versa,"7,9",573 expériences vécues,€ 126,"15e arr., Paris",4.0
1,Le Robinet d'Or,"9,0",646 expériences vécues,€ 184,"10e arr., Paris",3.0
4,The ReMIX Hotel,"8,4",1 116 expériences vécues,€ 99,"19e arr., Paris",4.0
5,TRIBE Paris Batignolles,"9,1",176 expériences vécues,€ 130,"17e arr., Paris",4.0
6,Hôtel Des Arts-Bastille,"7,7",854 expériences vécues,€ 125,"11e arr., Paris",NaN


In [82]:
dfhotels = dfhotels.dropna()
len(dfhotels)

725

In [ ]:
#dfhotels.to_excel('dfhotels.xlsx')

## 1.2.2. Scraping de la base de données "Notes"

On utilise des fonctions similaires à celles créées précédemment. 

D'abord, on récupère les url des pages de chaques hôtels à partir de la page de résultats. Les résulats sont stockés dans la liste 'href_hotels'.

Puis nous créons une boucle sur les url de la liste href_hotels afin de scraper pour chaque hôtel le détail de ses notes ainsi que son nom, et obtenir ainsi un dataframe concaténable à dfhotels.

In [87]:
def recup_href(page):
    href_hotel=[]
    liste = cree_liste(page)
    for page in liste: 
        #option1:
        #driver = webdriver.Chrome()
        #option2:
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(page)
        href_hotel+=[my_elem.get_attribute("href") for my_elem in wait(driver, 20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "h3._23bf57b84 > a")))]           
        driver.quit()
    return(href_hotel)

In [ ]:
#page réellement utilisée: 
#page = 'https://www.booking.com/searchresults.fr.html?label=gog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE&sid=29099b70bfcd7cf703ca2b9b7570611a&aid=356980&sb=1&sb_lp=1&src=theme_landing_index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fhotel%2Findex.fr.html%3Faid%3D356980%3Blabel%3Dgog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE%3Bsid%3D29099b70bfcd7cf703ca2b9b7570611a%3Bsrpvid%3D96be43728e9d017d%26%3B&ss=Paris%2C+%C3%8Ele-de-France%2C+France&is_ski_area=&checkin_year=2022&checkin_month=1&checkin_monthday=10&checkout_year=2022&checkout_month=1&checkout_monthday=11&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=paris&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1456928&dest_type=city&iata=PAR&place_id_lat=48.85668&place_id_lon=2.351476&search_pageview_id=813443bdfdfb00e8&search_selected=true&search_pageview_id=813443bdfdfb00e8&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0'
#page valide jusqu'au 31/01/2022:
page='https://www.booking.com/searchresults.fr.html?aid=356980&label=gog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE&sid=c81cc6cca40ba1a705208d4213ee6868&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D356980%3Blabel%3Dgog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE%3Bsid%3Dc81cc6cca40ba1a705208d4213ee6868%3Btmpl%3Dsearchresults%3Bcity%3D-1456928%3Bclass_interval%3D1%3Bdest_id%3D-1456928%3Bdest_type%3Dcity%3Blabel_click%3Dundef%3Boffset%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrpvid%3Db13d753bb2110028%3Bssb%3Dempty%26%3B&ss=Paris&is_ski_area=0&ssne=Paris&ssne_untouched=Paris&city=-1456928&checkin_year=2022&checkin_month=2&checkin_monthday=1&checkout_year=2022&checkout_month=2&checkout_monthday=2&group_adults=2&group_children=0&no_rooms=1&from_sf=1'
href_hotels=recup_href(page)

In [90]:
print(href_hotels[:3])

['https://www.booking.com/hotel/fr/acaciasetoile.fr.html?label=gog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE&sid=c81cc6cca40ba1a705208d4213ee6868&aid=356980&ucfs=1&arphpl=1&checkin=2022-02-01&checkout=2022-02-02&dest_id=-1456928&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=419b849a11880133&srepoch=1641408695&all_sr_blocks=5134903_261166977_2_2_0&highlighted_blocks=5134903_261166977_2_2_0&matching_block_id=5134903_261166977_2_2_0&sr_pri_blocks=5134903_261166977_2_2_0__12906&from=searchresults#hotelTmpl', 'https://www.booking.com/hotel/fr/comfort-paris-la-fayette.fr.html?label=gog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE&sid=c81cc6cca40ba1a705208d4213ee6868&aid=356980&ucfs=1&arphpl=1&checkin=2022-02-01&checkout=2022-02-02&

Les deux fonctions suivantes servent à la mise en forme des données scrapés. 
On les utilisera juste aprés dans la fonction recup_data2.

In [92]:
def mef_nom(nom):
    x=""
    for lettre in nom:
        x+=lettre
    return x[6:]

print(mef_nom(['H', 'ô', 't', 'e', 'l', ' ', 'H', 'o', 't', 'e', 'l', ' ', 'E', 'l', 'y', 's', 'e', 'e', 's', ' ', 'O', 'p', 'e', 'r', 'a']))

def mef_notes(notes):
    y=[]
    for note in notes:
        y.append(float(note.replace(',' , '.')))
    return y

print(mef_notes(['9,4', '8,1', '8,6', '8,6', '8,2', '8,9', '8,3']))

Hotel Elysees Opera
[9.4, 8.1, 8.6, 8.6, 8.2, 8.9, 8.3]


In [95]:
def recup_data2(href):
    hotel=[]
    #dans l'ordre d'apparition sur la page
    perso=[]
    equip=[]
    prop=[]
    conf=[]
    rap_qp=[]
    situ=[]
    wifi=[]
    
    for url in href :
        #option1:
        #driver = webdriver.Chrome()
        #option2:
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        nom=wait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='hp__hotel-title']/h2"))).text
        notes=[elem.get_attribute("innerHTML") for elem in driver.find_elements_by_xpath("//div[@class='c-score-bar']//span[@class='c-score-bar__score']")]
        #la liste des notes est dans l'ordre : perso,equip... 
        
        nom=mef_nom(nom)
        notes=mef_notes(notes)
        
        #Ajout aux listes globales 
        hotel.append(nom)
        perso.append(notes[0])
        equip.append(notes[1])
        prop.append(notes[2])
        conf.append(notes[3])
        rap_qp.append(notes[4])
        situ.append(notes[5])
        wifi.append(notes[6])  

        driver.quit()
    
    df = pd.DataFrame({'hotel':hotel,'perso':perso,'equip':equip,'prop':prop,'conf':conf,'rap_qp':rap_qp,'situ':situ,'wifi':wifi})
    
    return df

In [99]:
df1.head()

,hotel,perso,equip,prop,conf,rap_qp,situ,wifi
0,Acacias Etoile,9.1,9.3,8.9,8.3,8.7,8.3,8.6
1,Hôtel Paris La Fayette,8.8,8.8,8.3,8.3,8.1,7.7,8.4
2,Mary's Hotel République,8.5,9.0,7.6,8.0,7.4,7.3,8.5
3,Hotel de l'Aqueduc,8.6,7.4,8.1,7.8,7.8,7.6,8.3
4,Petit Madeleine Hôtel,8.5,9.2,9.1,8.2,8.7,8.4,8.8


In [ ]:
#df1.to_excel('notes_hotels.xlsx')

## 1.2.3. Associer les bases de données "Hôtels" et "Notes"

In [ ]:
df1=recup_data2(href_hotels)

**Si vous n'avez pas pu exécuter le code de scraping pour des raisons de temps, les lignes ci-dessous afin vous permettrons de suivre la suite du notebook.**

In [70]:
dfhotels = pd.read_excel('dfhotels.xlsx', index_col='Unnamed: 0')
notes_hotels = pd.read_excel('notes_hotels.xlsx', index_col='Unnamed: 0')

In [71]:
dfhotels.head()

,hotel,perso,equip,prop,conf,rap_qp,situ,wifi
0,Hôtel R de Paris - Boutique Hotel,9.0,9.5,9.5,9.3,8.9,8.7,9.1
1,Le 7 Eiffel Hotel,8.9,8.3,8.5,8.3,7.9,7.5,9.0
2,Petit Madeleine Hôtel,8.5,9.2,9.1,8.2,8.7,8.4,8.8
3,Hôtel Des Marronniers,9.1,9.7,8.8,8.2,8.5,8.1,9.1
4,Hotel des Nations Saint Germain,8.8,9.1,7.9,7.8,7.8,7.6,7.7


In [72]:
notes_hotels.head()

,hotel,perso,equip,prop,conf,rap_qp,situ,wifi
0,Hôtel R de Paris - Boutique Hotel,9.0,9.5,9.5,9.3,8.9,8.7,9.1
1,Le 7 Eiffel Hotel,8.9,8.3,8.5,8.3,7.9,7.5,9.0
2,Petit Madeleine Hôtel,8.5,9.2,9.1,8.2,8.7,8.4,8.8
3,Hôtel Des Marronniers,9.1,9.7,8.8,8.2,8.5,8.1,9.1
4,Hotel des Nations Saint Germain,8.8,9.1,7.9,7.8,7.8,7.6,7.7


Pour pouvoir unifier deux bases de données selon une même caractéristique (ici, le nom des hôtels), il faut que cette caractéristique soit présente dans chaque base de données sous la forme d'une colonne, et que cette colonne ait le même nom dans les deux bases. Ce n'est actuellement pas le cas. On renomme donc la colonne avant de faire la jointure. 

In [73]:
notes_hotels = notes_hotels.rename(columns={'hotel':'Hôtels'})
dfhotels_merged = pd.merge(dfhotels, notes_hotels, on='Hôtels')
dfhotels_merged.head()

,Hôtels,Note moyenne,Nombre de commentaires,Prix,Arrondissement,Etoiles,perso,equip,prop,conf,rap_qp,situ,wifi
0,Vice Versa,"7,9",573 expériences vécues,€ 126,"15e arr., Paris",4,8.1,8.7,8.3,8.1,7.9,7.5,8.8
1,Vice Versa,"7,9",573 expériences vécues,€ 126,"15e arr., Paris",4,8.1,8.7,8.3,8.1,7.9,7.5,8.8
2,The ReMIX Hotel,"8,4",1 116 expériences vécues,€ 99,"19e arr., Paris",4,7.8,8.8,8.8,8.8,8.4,8.3,8.3
3,TRIBE Paris Batignolles,"9,1",176 expériences vécues,€ 130,"17e arr., Paris",5,8.8,9.6,9.4,9.4,9.2,8.8,9.0
4,Hotel Aida Marais,"7,9",1 046 expériences vécues,€ 141,"10e arr., Paris",3,8.6,8.7,8.3,7.9,8.1,7.7,7.9


Il semble qu'il y ait encore des "duplicates". On va les supprimer.

In [76]:
dfhotels_merged = dfhotels_merged.drop_duplicates()
dfhotels_merged.dtypes

Hôtels                     object
Note moyenne               object
Nombre de commentaires     object
Prix                       object
Arrondissement             object
Etoiles                     int64
perso                     float64
equip                     float64
prop                      float64
conf                      float64
rap_qp                    float64
situ                      float64
wifi                      float64
dtype: object

Comme dans la base de données "Prix", les données ne sont pas exploitables telles quelles. On constate en effet que les colonnes "Note moyenne", "Nombre de commentaires", "Prix" et "Arrondissement" ne sont pas au format numérique. Avant de passer au format numérique, il va nous falloir enlever les chaînes de caractères, et donc convertir les colonnes au format 'str'.

In [77]:
dfhotels_merged["Note moyenne"] = dfhotels_merged["Note moyenne"].astype(str)
dfhotels_merged["Note moyenne"] = dfhotels_merged["Note moyenne"].str.replace(',', '.')
dfhotels_merged["Note moyenne"] = pd.to_numeric(dfhotels_merged["Note moyenne"])

dfhotels_merged["Prix"] = dfhotels_merged["Prix"].astype(str)
dfhotels_merged["Prix"] = dfhotels_merged["Prix"].str.replace('€ ', '')
dfhotels_merged["Prix"] = pd.to_numeric(dfhotels_merged["Prix"])

dfhotels_merged["Arrondissement"] = dfhotels_merged["Arrondissement"].astype(str)
dfhotels_merged["Arrondissement"] = dfhotels_merged["Arrondissement"].str.replace('e arr., Paris', '')
dfhotels_merged["Arrondissement"] = dfhotels_merged["Arrondissement"].str.replace('er arr., Paris', '')
dfhotels_merged["Arrondissement"] = pd.to_numeric(dfhotels_merged["Arrondissement"])

dfhotels_merged["Nombre de commentaires"] = dfhotels_merged["Nombre de commentaires"].astype(str)
dfhotels_merged["Nombre de commentaires"] = dfhotels_merged["Nombre de commentaires"].str.replace(' expériences vécues', '')
dfhotels_merged["Nombre de commentaires"] = dfhotels_merged["Nombre de commentaires"].str.replace(' ', '')
dfhotels_merged["Nombre de commentaires"] = pd.to_numeric(dfhotels_merged["Nombre de commentaires"])

In [78]:
dfhotels_merged.head()

,Hôtels,Note moyenne,Nombre de commentaires,Prix,Arrondissement,Etoiles,perso,equip,prop,conf,rap_qp,situ,wifi
0,Vice Versa,7.9,573,126,15,4,8.1,8.7,8.3,8.1,7.9,7.5,8.8
2,The ReMIX Hotel,8.4,1116,99,19,4,7.8,8.8,8.8,8.8,8.4,8.3,8.3
3,TRIBE Paris Batignolles,9.1,176,130,17,5,8.8,9.6,9.4,9.4,9.2,8.8,9.0
4,Hotel Aida Marais,7.9,1046,141,10,3,8.6,8.7,8.3,7.9,8.1,7.7,7.9
5,Hôtel Hor Les Lumières,8.6,972,117,11,4,9.4,8.8,9.0,8.3,8.5,8.5,8.5


In [ ]:
#dfhotels_merged.to_excel('dfhotels_merged.xlsx', index=False)

Sous cette forme, les données sont exploitables. L'analyse des deux bases ainsi obtenues (dfprix en 1.1 et dfhotels_merged en 1.2) est effectuée dans les deux autres fichiers du repository (Partie2 pour dfprix et Partie3 pour dfhotels_merged). 